# Import

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
torch.manual_seed(1)

# Softmax

In [3]:
z = torch.FloatTensor([1 , 2, 3])

In [29]:
hypothesis = F.softmax(z, dim=1)
print(hypothesis)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward0>)


In [30]:
hypothesis.sum()

tensor(3.0000, grad_fn=<SumBackward0>)

# Cross Entropy Loss(Low-level)

In [8]:
z= torch.rand(3, 5, requires_grad=True)
hypothesis = F.softmax(z, dim=1)
print(hypothesis)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward0>)


In [9]:
y = torch.randint(5, (3,)).long() # cloass = 5, samples = 3
print(y)

tensor([0, 2, 1])


In [24]:
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

In [25]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(1).mean()
print(cost)

tensor(1.4689, grad_fn=<MeanBackward0>)


# Cross Entropy Loss with torch.nn.functional

In [19]:
# Low level
torch.log(F.softmax(z, 1))

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward0>)

In [20]:
# High level
F.log_softmax(z, 1)

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward0>)

In [26]:
# Low level
(y_one_hot * -torch.log(F.softmax(z, 1))).sum(1).mean()

tensor(1.4689, grad_fn=<MeanBackward0>)

In [27]:
# High level
F.nll_loss(F.log_softmax(z, 1), y) # NLL = Negative Log Likelihood

tensor(1.4689, grad_fn=<NllLossBackward0>)

In [28]:
F.cross_entropy(z, y)

tensor(1.4689, grad_fn=<NllLossBackward0>)

# Training with Low-level Cross Entropy Loss

In [32]:
x_train = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]

y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [34]:
W = torch.zeros((4, 3), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

optimizer = optim.SGD([W, b], lr= 0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    hypothesis = F.softmax(x_train.matmul(W) + b, dim = 1)
    y_ont_hot = torch.zeros_like(hypothesis)
    y_ont_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_ont_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(1)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print('Epoch {:4d}  {} Cost: {:.5f}'.format(epoch, nb_epochs, cost.item()))

RuntimeError: grad can be implicitly created only for scalar outputs

# Training with F.cross_entropy

In [35]:
W = torch.zeros((4, 3), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

optimizer = optim.SGD([W, b], lr= 0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    z = x_train.matmul(W) + b
    cost = F.cross_entropy(z, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print('Epoch {:4d}  {} Cost: {:.5f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0  1000 Cost: 1.09861
Epoch  100  1000 Cost: 0.76105
Epoch  200  1000 Cost: 0.68999
Epoch  300  1000 Cost: 0.64323
Epoch  400  1000 Cost: 0.60412
Epoch  500  1000 Cost: 0.56826
Epoch  600  1000 Cost: 0.53392
Epoch  700  1000 Cost: 0.50029
Epoch  800  1000 Cost: 0.46691
Epoch  900  1000 Cost: 0.43351
Epoch 1000  1000 Cost: 0.39996


# High-level Implementation with nn.Module

In [40]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3)
        
    def forward(self, x):
        return self.linear(x)

In [41]:
model = SoftmaxClassifierModel()

In [42]:
optimizer = optim.SGD(model.parameters(), lr= 0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    prediction = model(x_train)
    
    cost = F.cross_entropy(prediction, y_train)
    
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print('Epoch {:4d}  {} Cost: {:.5f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0  1000 Cost: 1.84572
Epoch  100  1000 Cost: 0.64715
Epoch  200  1000 Cost: 0.56887
Epoch  300  1000 Cost: 0.51570
Epoch  400  1000 Cost: 0.47173
Epoch  500  1000 Cost: 0.43249
Epoch  600  1000 Cost: 0.39588
Epoch  700  1000 Cost: 0.36051
Epoch  800  1000 Cost: 0.32523
Epoch  900  1000 Cost: 0.28922
Epoch 1000  1000 Cost: 0.25409
